<a href="https://colab.research.google.com/github/armiro/Dental-Caries-Classification/blob/master/periapical_dental_caries_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, BatchNormalization, Add
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import glob, time, os, shutil

In [0]:
path = './drive/My Drive/periapical_dataset/healthy/**.jpg'
neg_images = list()
for img_name in glob.glob(pathname=path):
  img = load_img(path=img_name, color_mode='grayscale')
  img = img_to_array(img=img, data_format='channels_last')
  # img = np.expand_dims(a=img, axis=-1)
  neg_images.append(img)

neg_images = np.array(neg_images)
print('num healthy images:', len(neg_images))

num healthy images: 139


In [0]:
path = './drive/My Drive/periapical_dataset/caries/**.jpg'
pos_images = list()
for img_name in glob.glob(pathname=path):
  img = load_img(path=img_name, color_mode='grayscale')
  img = img_to_array(img=img, data_format='channels_last')
  # img = np.expand_dims(a=img, axis=-1)
  pos_images.append(img)

pos_images = np.array(pos_images)
print('num carious images:', len(pos_images))

num carious images: 145


In [0]:
neg_labels = [0 for _ in range(len(neg_images))]
pos_labels = [1 for _ in range(len(pos_images))]

In [0]:
X = np.concatenate((pos_images, neg_images))
y = np.array(pos_labels + neg_labels)

print('num total dataset images:', len(X))
print('num total dataset labels:', len(y))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rescale=1./255, rotation_range=0, zoom_range=[1., 2.], 
                         width_shift_range=0.25, height_shift_range=0.25, 
                         shear_range=0.15, horizontal_flip=True, 
                         vertical_flip=True, fill_mode='nearest')

test_aug = ImageDataGenerator(rescale=1./255)

In [0]:
from keras import Model
inputA = Input(shape=X_train[0].shape)

x = Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu')(inputA)
x = BatchNormalization()(x)
x1 = MaxPooling2D(pool_size=2, strides=2, padding='same')(x)
x2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(x1)
x2 = BatchNormalization()(x2)
x3 = MaxPooling2D(pool_size=2, strides=2, padding='same')(x2)
x4 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(x3)
x4 = BatchNormalization()(x4)
x5 = MaxPooling2D(pool_size=2, strides=2, padding='same')(x4)
x6 = Flatten()(x5)
x7 = Dense(units=50, activation='relu')(x6)
x8 = Dense(units=25, activation='relu')(x7)
x9 = Dense(units=10, activation='relu')(x8)
x10 = Dense(units=1, activation='sigmoid')(x9)
net = Model(inputs=inputA, outputs=x10)

In [19]:
net.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
net.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 500, 500, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 250, 250, 128)     1280      
_________________________________________________________________
batch_normalization_4 (Batch (None, 250, 250, 128)     512       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 125, 125, 128)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 63, 63, 64)        73792     
_________________________________________________________________
batch_normalization_5 (Batch (None, 63, 63, 64)        256       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 64)        0   

In [20]:
# find last saved weights file and its epoch number
weights_folder = '/content/drive/My Drive/model'
num_epochs = list()
for weights_file in glob.glob(weights_folder + '/**.hdf5'):
  num_epoch = int(weights_file[weights_file.find('=')+1:weights_file.find('_')])
  num_epochs.append((num_epoch, weights_file))

next_init_epoch = max(num_epochs)[0]
print('last saved epoch:', next_init_epoch)
last_epoch_weights_file = max(num_epochs)[1]
print('last saved file:', last_epoch_weights_file)

# load the last saved weigths file as the model's weights
net.load_weights(last_epoch_weights_file)

# delete all weigths files saved before
for weights_file in os.listdir(weights_folder):
    file_path = os.path.join(weights_folder, weights_file)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('failed to delete %s. because of: %s' % (file_path, e))

last saved epoch: 683
last saved file: /content/drive/My Drive/model/epochs=683_acc=0.9133.hdf5


In [0]:
from keras.callbacks import ModelCheckpoint

file_path = '/content/drive/My Drive/model/epochs={epoch:03d}_acc={acc:.4f}.hdf5'
checkpoint = ModelCheckpoint(file_path, monitor='acc', verbose=1,
                             save_best_only=True, mode='max')

cb_list = [checkpoint]

In [22]:
net.fit_generator(generator=aug.flow(x=X_train, y=y_train, batch_size=16), 
                  steps_per_epoch=50, initial_epoch=next_init_epoch, epochs=999, 
                  verbose=1, callbacks=cb_list)


Epoch 684/999
50/50 [==============================] - 16s 321ms/step - loss: 0.2254 - acc: 0.9046

Epoch 00684: acc improved from -inf to 0.90539, saving model to /content/drive/My Drive/model/epochs=684_acc=0.9054.hdf5
Epoch 685/999
50/50 [==============================] - 14s 288ms/step - loss: 0.2904 - acc: 0.8839

Epoch 00685: acc did not improve from 0.90539
Epoch 686/999
50/50 [==============================] - 16s 310ms/step - loss: 0.2511 - acc: 0.8883

Epoch 00686: acc did not improve from 0.90539
Epoch 687/999
50/50 [==============================] - 16s 312ms/step - loss: 0.2262 - acc: 0.9112

Epoch 00687: acc improved from 0.90539 to 0.90670, saving model to /content/drive/My Drive/model/epochs=687_acc=0.9067.hdf5
Epoch 688/999
50/50 [==============================] - 15s 301ms/step - loss: 0.3000 - acc: 0.8686

Epoch 00688: acc did not improve from 0.90670
Epoch 689/999
50/50 [==============================] - 16s 310ms/step - loss: 0.2740 - acc: 0.8887

Epoch 00689: acc 